# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

In [1]:
%store -r train_data_set
%store -r train_labels
%store -r test_data_set
%store -r test_labels

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

Is this equation correct?

In [2]:
import numpy as np

In [3]:
def build_kernel(data_set, kernel_type='linear',power=1):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        objects_count = len(data_set)
        kernel = np.dot(data_set, data_set.T)**power # it is not the equation from above
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
def choose_set_for_label(data_set, labels, label):
    train_data_set, test_data_set, train_labels, test_labels = train_test_split(data_set, np.where(labels == label, 1, -1), test_size=0.2, random_state = 1)
    return train_data_set, test_data_set, train_labels, test_labels

In [6]:
def get_labels_count(labels):
    return len(np.unique(labels))

Use the code that we have implemented earlier:

In [7]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)
    objects_count = len(train_data_set)
    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [8]:
import cvxopt
from sklearn.metrics import accuracy_score

In [9]:
from sklearn.datasets import load_iris
iris = load_iris()
data_set = iris.data
labels = iris.target

In [10]:
for label in np.unique(labels):
    train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set,labels,label)
    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
    predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
    predicted = list(predicted.astype(int))
    print(accuracy_score(predicted, test_labels))

     pcost       dcost       gap    pres   dres
 0:  1.1546e+02 -1.8010e+03  4e+03  2e-01  2e-15
 1:  7.6936e+01 -1.7425e+02  3e+02  6e-03  2e-15
 2:  1.0331e+01 -2.2446e+01  3e+01  2e-16  3e-15
 3: -4.5784e-01 -5.0020e+00  5e+00  5e-16  2e-15
 4: -1.4185e+00 -2.5432e+00  1e+00  2e-16  5e-16
 5: -1.6791e+00 -2.1523e+00  5e-01  2e-16  2e-16
 6: -1.7984e+00 -2.0190e+00  2e-01  2e-16  2e-16
 7: -1.8495e+00 -1.9782e+00  1e-01  2e-16  2e-16
 8: -1.8855e+00 -1.9011e+00  2e-02  3e-16  2e-16
 9: -1.8917e+00 -1.8920e+00  2e-04  2e-16  2e-16
10: -1.8918e+00 -1.8918e+00  2e-06  7e-16  2e-16
11: -1.8918e+00 -1.8918e+00  2e-08  7e-16  2e-16
Optimal solution found.
0.36666666666666664
     pcost       dcost       gap    pres   dres
 0:  1.1388e+02 -2.4345e+03  5e+03  2e-01  2e-15
 1:  9.1003e+01 -2.2390e+02  4e+02  6e-03  3e-15
 2:  1.2955e+01 -2.6102e+01  4e+01  3e-05  3e-15
 3: -1.0639e-01 -5.5584e+00  5e+00  8e-16  2e-15
 4: -1.3644e+00 -2.2545e+00  9e-01  2e-16  5e-16
 5: -1.5969e+00 -2.0306e+00